In [57]:
import requests as r
import pymysql as mysql
import datetime
import time


In [68]:
import pymysql as mysql

class league_match(object):
    
    def __init__(self,journey,homeTeam,homeTeam_id,awayTeam,awayTeam_id,date,goalsAwayTeam,goalsHomeTeam,league_id):
        self.journey=journey
        self.homeTeam=homeTeam
        self.awayTeam=awayTeam
        self.homeTeam_id=homeTeam_id
        self.awayTeam_id=awayTeam_id
        self.date=date
        self.goalsAwayTeam=goalsAwayTeam
        self.goalsHomeTeam=goalsHomeTeam
        self.league_id=league_id
        self.index=str(journey)+str(homeTeam_id)+str(awayTeam_id)+str(league_id)

class league(object):
    
    def __init__(self,league_id):
        
        self.clubs={}
        self.matchs=[]
        self.league_id=league_id
        self.get_list_team()
        self.get_matchs()
        
    def add_club_name(self,club_name,club_id):
        
        self.clubs[club_name]=club_id
        
        
    def add_match(self,match):
        
        self.matchs.append(match)
        
        
    def get_list_team(self):
        
        key="key"
        url="http://api.football-data.org/v1/soccerseasons/"+str(self.league_id)+"/teams"
        headers = {"X-Auth-Token": key}
        data=r.get(url,headers=headers).json()
        
        for d in data["teams"]:
            id_=d["_links"]["self"]["href"][-3:]
            league.add_club_name(self,club_name=d["name"],club_id=id_)
            
       
    def get_matchs(self):
        
        key="key"
        url='http://api.football-data.org/v1/soccerseasons/'+str(self.league_id)+'/fixtures'
        headers = {"X-Auth-Token": key}
        data=r.get(url,headers=headers)
        data=data.json()
        for d in data['fixtures']:
            
            if d["status"]=="FINISHED":
                league.add_match(self,league_match(journey=d["matchday"]
                                        ,homeTeam=d["homeTeamName"]
                                        ,awayTeam=d["awayTeamName"]
                                        ,homeTeam_id=d["_links"]["homeTeam"]["href"][-3:]
                                        ,awayTeam_id=d["_links"]["awayTeam"]["href"][-3:]     
                                        ,date=d["date"][:10]
                                        ,goalsAwayTeam=d["result"]["goalsAwayTeam"]
                                        ,goalsHomeTeam=d["result"]["goalsHomeTeam"]
                                        ,league_id=self.league_id))

class amz_rds(object):
    
    
    def __init__(self,host,username,password,db,port):
        
        
        self.host=host
        self.username=username
        self.password=password
        self.db=db
        self.port=port
        self.cnx=None
        
    def open_cnx(self,state):
         
        if state==True:
            
            self.cnx=mysql.Connect(self.host,self.username,self.password, self.db,self.port)
        
        elif state==False:
            
            self.cnx=mysql.Connect(self.host,self.username,self.password, self.db,self.port).close()
            
    def add_match(self,cnx,journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_):
        
        cursor = cnx.cursor()
        
        add_match = ("INSERT INTO matchs_results "
                   "(journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_) "
                   "VALUES (%s, %s,%s, %s,%s, %s,%s)"
                    "ON DUPLICATE KEY UPDATE "
                    "journey=Values(journey),"
                    "homeTeam_id=VALUES(homeTeam_id),"
                    "awayTeam_id=VALUES(awayTeam_id),"
                    "goalsHomeTeam=VALUES(goalsHomeTeam),"
                    "goalsAwayTeam=VALUES(goalsAwayTeam),"
                    "date=VALUES(date),"
                    "index_=Values(index_);")   

        data_match = (journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_)

        cursor.execute(add_match, data_match)

        emp_no = cursor.lastrowid

        cnx.commit()

        cursor.close()
        
    def add_all_match(self,league):
        
        for m in league.matchs:
            
            self.add_match(self.cnx,m.journey,m.homeTeam_id,m.awayTeam_id,m.goalsHomeTeam,m.goalsAwayTeam,m.date,m.index)

            
            